In [1]:
import sys, os

import torch

from PIL import Image
import torchvision.transforms as transforms
import os

import numpy as np
import cv2

import argparse

external_repo_path = os.path.abspath('../../ide/external/transformer_explainability')

if external_repo_path not in sys.path:
    sys.path.append(external_repo_path)
sys.path.append(os.path.abspath(".."))

from baselines.ViT.ViT_LRP import vit_base_patch16_224 as vit_LRP
from baselines.ViT.ViT_explanation_generator import LRP

from external.pruning_by_explaining.my_datasets import WaterBirds, get_sample_indices_for_group, WaterBirdSubset, ISIC, ISICSubset


In [2]:
d = {
    "isic_30_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/isic_30_3_r0.96.pth",
    "isic_30_0123_minus_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/isic_30_0123_minus_3_r0.96.pth",
    "wb_30_12": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_12_r0.1.pth",
    "wb_30_0123_minus_2_div_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_0123_minus_2_div_2_r0.3.pth",
    "wb_30_0123_minus_2_div_2_0.1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_0123_minus_2_div_2_r0.1.pth",
    "wb_30_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_0123_r0.1.pth",
    "wb_30_0123_minus_1_div_2_0.1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_0123_minus_1_div_2_r0.1.pth",
    "wb_attn_30_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_attn_30_0123_r0.3.pth",
    "wb_attn_30_0123_0.1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_attn_30_0123_r0.1.pth",
    "wb_attn_30_0123_minus_1_div_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/checkpoints/wb_30_0123_minus_1_div_2_r0.1.pth",

    
    "wb_linear_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Linear_g-0_1_2_3_n-100_seed-1/model_pruned_linear_r0.4.pth",
    "wb_linear_1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Linear_g-1_n-100_seed-1/model_pruned_linear_r0.4.pth",
    "wb_linear_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Linear_g-2_n-100_seed-1/model_pruned_linear_r0.4.pth",
    "wb_linear_counter_1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Linear_g-0_1_2_3_minus-1_n-100_seed-2/model_pruned_linear_r0.4.pth",
    "wb_linear_counter_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Linear_g-0_1_2_3_minus-1_n-100_seed-3/model_pruned_linear_r0.4.pth",
    
    
    
    "wb_softmax_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Softmax_g-0_1_2_3_n-100_seed-1/model_pruned_softmax_r0.5.pth",
    "wb_softmax_1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Softmax_g-1_n-100_seed-1/model_pruned_softmax_r0.6.pth",    
    "wb_softmax_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Softmax_g-2_n-100_seed-1/model_pruned_softmax_r0.4.pth",
    "wb_softmax_counter_1": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Softmax_g-0_1_2_3_minus-1_n-100_seed-2/model_pruned_softmax_r0.6.pth",
    "wb_softmax_counter_2": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/wb_Softmax_g-0_1_2_3_minus-2_n-100_seed-2/model_pruned_softmax_r0.5.pth",

    "isic_linear_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Linear_g-0_1_2_3_n-100_seed-1/model_pruned_linear_r0.98.pth",
    "isic_linear_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Linear_g-3_n-100_seed-1/model_pruned_linear_r0.98.pth",
    "isic_linear_counter_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Linear_g-0_1_2_3_minus-3_n-100_seed-3/model_pruned_linear_r0.96.pth",

    "isic_softmax_0123": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Softmax_g-0_1_2_3_n-100_seed-1/model_pruned_softmax_r0.9.pth",
    "isic_softmax_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Softmax_g-3_n-100_seed-1/model_pruned_softmax_r0.92.pth",
    "isic_softmax_counter_3": "/home/primmere/ide/external/pruning_by_explaining/my_experiments/f_experiments/isic_Softmax_g-0_1_2_3_minus-3_n-100_seed-3/model_pruned_softmax_r0.9.pth",

}

In [3]:
dataset="isic"
pruned = True
convert_bool = True
n = 50
model_name = 'isic_softmax_counter_3'
groups = [0,1,2,3]






chkpt_path = d[model_name]

if dataset == "wb":
    if pruned:
        chkpt_path = chkpt_path
    else:
        chkpt_path = "/home/primmere/ide/dfr/logs/vit_waterbirds.pth"
if dataset == "isic":
    if pruned:
        chkpt_path = chkpt_path
    else:
        chkpt_path = '/home/primmere/logs/isic_logs_4/vit_isic_v2.pt'


if dataset == "wb":
    waterbirds = WaterBirds('/scratch_shared/primmere/waterbird', seed = 1, num_workers = 8)
    train = waterbirds.get_train_set()
    chosen_set = train
if dataset == "isic":
    isic = ISIC(
        "/scratch_shared/primmere/isic/isic_224/raw_224_with_selected", 
        metadata_path='/scratch_shared/primmere/isic/metadata_w_split_v2_w_elmos_modifications.csv', 
        seed=1, 
        num_workers=8
            )
    val = isic.get_valid_set()
    chosen_set = val


indices = get_sample_indices_for_group(chosen_set, n, "cuda", groups)
if dataset == "wb":
    train_subset = WaterBirdSubset(chosen_set, indices)
if dataset == "isic":
    train_subset = ISICSubset(chosen_set, indices)
train_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=n*len(groups), shuffle=False, num_workers=8)
x, y, g = next(iter(train_dataloader))

[0 1 2 3]
Number of unique labels: 2, Number of unique places: 2, Total groups: 4
group 0: 60
group 1: 60
group 2: 60
group 3: 60
target groups: [0, 1, 2, 3]


In [4]:
len(indices)

200

In [5]:
import re
from typing import Dict, Any

import torch


def convert(sd: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    """
    Convert a ViT-style checkpoint that uses:
      - LayerNorm stored under *.ln_*.module.{weight,bias}
      - Attention split into q/k/v submodules with *.{q,k,v}_proj.proj_{weight,bias}
      - Out proj under *.out_proj.proj_{weight,bias}
    into a checkpoint that uses:
      - *.ln_*.{weight,bias}  (no '.module')
      - Self-attention fused 'in_proj_{weight,bias}' with Q,K,V concatenated on dim=0
      - Out proj under *.out_proj.{weight,bias}

    All other keys are copied as-is.

    Args:
        sd: source state_dict from "checkpoint 2".

    Returns:
        A new state_dict matching "checkpoint 1" naming/shapes.
    """
    new_sd: Dict[str, torch.Tensor] = {}

    # 1) Copy keys while normalizing simple renames and skipping q/k/v parts
    for k, v in sd.items():
        # Strip the extra '.module' in layer norms (per-layer and final encoder.ln)
        k2 = (k.replace(".ln_1.module.", ".ln_1.")
                .replace(".ln_2.module.", ".ln_2.")
                .replace("encoder.ln.module.", "encoder.ln."))

        # Rename out proj from *.proj_weight/bias -> *.weight/bias
        k2 = (k2.replace(".self_attention.out_proj.proj_weight", ".self_attention.out_proj.weight")
                .replace(".self_attention.out_proj.proj_bias", ".self_attention.out_proj.bias"))

        # Skip q/k/v pieces; we'll fuse them after this loop.
        if (".self_attention.q_proj.proj_" in k
                or ".self_attention.k_proj.proj_" in k
                or ".self_attention.v_proj.proj_" in k):
            continue

        # Regular copy (clone to avoid accidental aliasing)
        new_sd[k2] = v.clone() if isinstance(v, torch.Tensor) else v

    # 2) Build fused in_proj_{weight,bias} per encoder layer
    #    Detect all layer indices that have q_proj weights.
    layer_ids = set()
    for k in sd.keys():
        m = re.match(r"encoder\.layers\.encoder_layer_(\d+)\.self_attention\.q_proj\.proj_weight", k)
        if m:
            layer_ids.add(int(m.group(1)))

    for i in sorted(layer_ids):
        base = f"encoder.layers.encoder_layer_{i}.self_attention"

        # Fetch q/k/v weights & biases
        qw = sd[f"{base}.q_proj.proj_weight"]
        kw = sd[f"{base}.k_proj.proj_weight"]
        vw = sd[f"{base}.v_proj.proj_weight"]

        qb = sd[f"{base}.q_proj.proj_bias"]
        kb = sd[f"{base}.k_proj.proj_bias"]
        vb = sd[f"{base}.v_proj.proj_bias"]

        # Sanity checks (optional but helpful)
        assert qw.shape == kw.shape == vw.shape, f"q/k/v weight shapes differ at layer {i}"
        assert qb.shape == kb.shape == vb.shape, f"q/k/v bias shapes differ at layer {i}"
        assert qw.dim() == 2 and qb.dim() == 1, f"unexpected dims at layer {i}"

        # Concatenate in [Q; K; V] order along output-dim (dim=0)
        in_w = torch.cat([qw, kw, vw], dim=0)  # [3*D, D]
        in_b = torch.cat([qb, kb, vb], dim=0)  # [3*D]

        new_sd[f"{base}.in_proj_weight"] = in_w
        new_sd[f"{base}.in_proj_bias"] = in_b

    return new_sd

In [6]:
import os
import numpy as np
from PIL import Image

def _to_pil(img):
    """
    Convert a variety of common image formats (PIL, np.ndarray, torch.Tensor)
    into a PIL.Image without changing the spatial content.

    Assumptions handled:
    - Channel-first arrays/tensors (C,H,W) with C in {1,3}
    - Channel-last arrays (H,W,C)
    - Float data in [0,1] or [-1,1] (auto-clipped to [0,1])
    - uint8 arrays already in [0,255]
    """
    try:
        import torch
        is_tensor = isinstance(img, torch.Tensor)
    except Exception:
        is_tensor = False

    if isinstance(img, Image.Image):
        return img

    if is_tensor:
        img = img.detach().cpu().numpy()

    if isinstance(img, np.ndarray):
        arr = img

        # Move channel-first -> channel-last if needed
        if arr.ndim == 3 and arr.shape[0] in (1, 3) and arr.shape[0] != arr.shape[-1]:
            arr = np.transpose(arr, (1, 2, 0))

        # If single-channel with shape (H,W,1), squeeze to (H,W)
        if arr.ndim == 3 and arr.shape[-1] == 1:
            arr = arr[..., 0]

        # Normalize to uint8 if float
        if np.issubdtype(arr.dtype, np.floating):
            # Handle possible [-1,1] range by clipping to [0,1]
            arr = np.clip(arr, 0.0, 1.0)
            arr = (arr * 255.0 + 0.5).astype(np.uint8)
        elif arr.dtype != np.uint8:
            # Generic cast (e.g., int16) with clipping
            info = np.iinfo(arr.dtype) if np.issubdtype(arr.dtype, np.integer) else None
            if info is not None:
                arr = np.clip(arr, info.min, info.max)
                # Try to scale to 0..255 if not already in that range
                arr = arr.astype(np.float32)
                arr -= arr.min()
                maxv = arr.max() if arr.max() != 0 else 1.0
                arr = (arr / maxv * 255.0 + 0.5).astype(np.uint8)
            else:
                # Fallback
                arr = arr.astype(np.uint8)

        return Image.fromarray(arr)

    raise TypeError(f"Unsupported image type: {type(img)}")


def save_original_image(original_image, filename, out_dir):
    """
    Save the original (pre-heatmap) image into a sibling folder named 'orig_imgs'
    using the SAME filename you use for the heatmap (e.g., '{group}_{ID}.png').

    Parameters
    ----------
    original_image : PIL.Image | np.ndarray | torch.Tensor
        The raw/original image before overlays.
    filename : str
        The exact filename you use for the heatmap image (e.g., f"{group}_{sample_id}.png").
    out_dir : str | Path
        The folder where you currently save your heatmaps. 'orig_imgs' will be created inside this folder.
    """
    orig_dir = os.path.join(out_dir, "orig_imgs")
    os.makedirs(orig_dir, exist_ok=True)

    save_path = os.path.join(orig_dir, filename)
    pil_img = _to_pil(original_image)
    pil_img.save(save_path)
    return save_path


In [7]:
checkpoint = torch.load(chkpt_path, map_location=torch.device('cuda'), weights_only=False)
if pruned and convert_bool:
    checkpoint = convert(checkpoint)
model = vit_LRP(pretrained=True, num_classes=2)

new_checkpoint = {}

# fix ckpt to be suitable for their model
for k, v in checkpoint.items():
    new_key = k
    new_key = new_key.replace('class_token', 'cls_token')
    new_key = new_key.replace('conv_proj', 'patch_embed.proj')
    new_key = new_key.replace('encoder.pos_embedding', 'pos_embed')
    new_key = new_key.replace('encoder.ln.weight', 'norm.weight')
    new_key = new_key.replace('encoder.ln.bias', 'norm.bias')
    new_key = new_key.replace('heads.head.weight', 'head.weight')
    new_key = new_key.replace('heads.head.bias', 'head.bias')

    new_key = new_key.replace('encoder.layers.encoder_layer_', 'blocks.')
    new_key = new_key.replace('.ln_1', '.norm1')
    new_key = new_key.replace('.ln_2', '.norm2')
    new_key = new_key.replace('.self_attention.in_proj_weight', '.attn.qkv.weight')
    new_key = new_key.replace('.self_attention.in_proj_bias', '.attn.qkv.bias')
    new_key = new_key.replace('.self_attention.out_proj.weight', '.attn.proj.weight')
    new_key = new_key.replace('.self_attention.out_proj.bias', '.attn.proj.bias')
    new_key = new_key.replace('.mlp.0', '.mlp.fc1')
    new_key = new_key.replace('.mlp.3', '.mlp.fc2')

    new_checkpoint[new_key] = v

model.load_state_dict(new_checkpoint)
model = model.cuda()

attribution_generator = LRP(model)

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def tensorize(img_path):
    image = Image.open(img_path).convert('RGB')
    return preprocess(image)


def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return cam


def generate_visualization(original_image, class_index=None):
    transformer_attribution = attribution_generator.generate_LRP(original_image.unsqueeze(0).cuda(),
                                                                 method="transformer_attribution",
                                                                 index=class_index).detach()
    transformer_attribution = transformer_attribution.reshape(1, 1, 14, 14)
    transformer_attribution = torch.nn.functional.interpolate(transformer_attribution, scale_factor=16, mode='bilinear')
    transformer_attribution = transformer_attribution.reshape(224, 224).data.cpu().numpy()
    transformer_attribution = (transformer_attribution - transformer_attribution.min()) / (
            transformer_attribution.max() - transformer_attribution.min())
    # flip values
    transformer_attribution = 1 - transformer_attribution

    image_transformer_attribution = original_image.permute(1, 2, 0).data.cpu().numpy()
    image_transformer_attribution = (image_transformer_attribution - image_transformer_attribution.min()) / (
            image_transformer_attribution.max() - image_transformer_attribution.min())
    vis = show_cam_on_image(image_transformer_attribution, transformer_attribution)
    vis = np.uint8(255 * vis)
    vis = cv2.cvtColor(np.array(vis), cv2.COLOR_RGB2BGR)
    return vis


for i in range(len(x)):
    idx = i
    img = x[idx]

    if pruned:
        output_dir = f"imgs_{dataset}_pruned_{model_name}"
    else:
        output_dir = f"imgs_{dataset}"
    os.makedirs(output_dir, exist_ok=True)

    # check if model is correct
    logits = model(x[idx].unsqueeze(0).cuda())
    pred = torch.argmax(logits, dim=1)
    if pred == y[idx].item():
        corr = "c"
    else:
        corr = "w"
    
    img_name = f"group-{g[idx]}-{indices[idx]}"
    save_original_image(img, img_name+".png", "")
    # Generate visualization for the class predicted by the model (e.g., 'land')
    cat = generate_visualization(img, class_index=0)
    land_filename = img_name + f"_0_{corr}.png"
    cat_output_path = os.path.join(output_dir, land_filename)
    cv2.imwrite(cat_output_path, cat)
    
    # Generate visualization for class index 1 (e.g., 'water')
    dog = generate_visualization(img, class_index=1)
    water_filename = img_name + f"_1_{corr}.png"
    dog_output_path = os.path.join(output_dir, water_filename)
    cv2.imwrite(dog_output_path, dog)
    
    print(f"Visualization for '0' saved at: {cat_output_path}")  # cat
    print(f"Visualization for '1' saved at: {dog_output_path}")  # dog


Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-85_0_c.png
Visualization for '1' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-85_1_c.png
Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-61_0_c.png
Visualization for '1' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-61_1_c.png
Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-74_0_c.png
Visualization for '1' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-74_1_c.png
Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-71_0_c.png
Visualization for '1' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-71_1_c.png
Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-95_0_c.png
Visualization for '1' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-95_1_c.png
Visualization for '0' saved at: imgs_isic_pruned_isic_softmax_counter_3/group-0-88_0_c.png